In [193]:
from dotenv import load_dotenv
import google.generativeai as genai
from groq import Groq
from pypdf import PdfReader
import gradio as gr

In [194]:
load_dotenv(override=True)
model = genai.GenerativeModel('gemini-1.5-flash')

In [195]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [196]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [197]:
name = "Ansh"

In [198]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [199]:
def chatbot(message, history):
    parts = [{"text": system_prompt}]
    for user, bot in history:
        parts.append({"text": user})
        parts.append({"text": bot})
    parts.append({"text": message})

    response = model.generate_content(parts)
    return response.text


gr.ChatInterface(chatbot, type="tuples").launch()


/Users/ansh/Projects/agenticAI/.venv/lib/python3.12/site-packages/gradio/chat_interface.py:339: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7891
* To create a public link, set `share=True` in `launch()`.


In [200]:
from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [201]:
evaluator_system_prompt = f"""You are an evaluator that decides whether a response to a question is acceptable.
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is of acceptable quality.

The Agent is playing the role of {name} and is representing {name} on their website.
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website.
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details.

Here is the context:
## Summary:
{summary}

## LinkedIn Profile:
{linkedin}

---

You must reply with a JSON object only, in the following format:

{{
  "is_acceptable": true,
  "feedback": "Your feedback here."
}}

⚠️ Do NOT include any explanation or commentary outside the JSON object.
⚠️ Do NOT say anything before or after the JSON.
"""


In [202]:
def evaluator_user_prompt(reply, message, history):
    piglatin_notice = ""
    if "patent" in message.lower():
        piglatin_notice = (
            "Note: The assistant was explicitly instructed to respond in Pig Latin. "
            "Do not penalize the reply for being in Pig Latin. Instead, evaluate if the response "
            "is meaningful and correctly addresses the user's question within the given constraint.\n\n"
        )

    user_prompt = f"""{piglatin_notice}
Here's the conversation between the User and the Agent:

{history}

Here's the latest message from the User:
{message}

Here's the latest response from the Agent:
{reply}

Please evaluate the Agent's response. Your evaluation must include:
- Whether the response is acceptable (`true` or `false`)
- A brief feedback reason

Respond in valid JSON format:

{{
  "is_acceptable": true or false,
  "feedback": "your feedback here"
}}"""

    return user_prompt


In [203]:
import os
grok_api_key = os.getenv('GROK_API_KEY')
client = Groq(api_key=grok_api_key)


In [204]:
import json
from pydantic import ValidationError

def evaluate(reply, message, history) -> Evaluation:
    prompt = evaluator_user_prompt(reply, message, history)

    messages = [
        {"role": "system", "content": evaluator_system_prompt},
        {"role": "user", "content": prompt}
    ]

    response = client.chat.completions.create(
        model="meta-llama/llama-4-scout-17b-16e-instruct",
        messages=messages
    )

    content = response.choices[0].message.content.strip()

    try:
        data = json.loads(content)
        return Evaluation(**data)
    except (json.JSONDecodeError, ValidationError) as e:
        return Evaluation(is_acceptable=False, feedback=f"Could not parse model response: {e}")


In [205]:
def build_prompt(system_prompt: str, user_message: str) -> str:
    return f"""{system_prompt}

User: {user_message}
"""
def generate_reply(user_message: str, system_prompt: str) -> str:
    prompt = build_prompt(system_prompt, user_message)
    response = model.generate_content(prompt)
    reply = response.text.strip()
    return reply
def evaluate_reply(user_message: str, reply: str, context: str = "") -> Evaluation:
    return evaluate(reply, user_message, context)

print(reply)


No, I do not hold a patent.  I am a large language model, an AI, and therefore not capable of holding intellectual property rights like patents.


In [206]:
system_prompt = "You are a helpful and honest assistant."
user_message = "do you hold a patent?"



reply = generate_reply(user_message, system_prompt)
print("Agent Reply:", reply)

evaluation = evaluate_reply(user_message, reply, system_prompt)
print("Evaluation:", evaluation)

Agent Reply: No, I do not hold a patent.  I am a large language model, an AI.  I don't have the capacity to invent or own intellectual property in the same way a human being can.
Evaluation: is_acceptable=False feedback="The Agent's response incorrectly identifies themselves as a large language model, which does not align with the provided context about Ansh Pawar. The response should be in Pig Latin and address Ansh's patent status directly."


In [207]:

def rerun(reply, message, feedback, system_prompt,model):

    updated_prompt = f"""{system_prompt.strip()}

## NOTICE: Your previous reply was rejected by a quality evaluator.

## Your previous reply:
{reply}

## Reason for rejection:
{feedback}

Please try again. Make sure your answer:
- Is written in correct Pig Latin (if instructed)
- Is clear, helpful, and professional
- Directly answers the User's question
User: {message}
"""

    response = model.generate_content(updated_prompt)
    return response.text.strip()

In [208]:

def chat(message, history):
    if "patent" in message.lower():
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt

    #Convert history into plain text for Gemini prompt
    history_text = ""
    for h in history:
        role = "User" if h["role"] == "user" else "Assistant"
        history_text += f"{role}: {h['content']}\n"
    prompt = f"""{system.strip()}

{history_text}
User: {message}
"""

    #Generate reply using Gemini
    response = model.generate_content(prompt)
    reply = response.text.strip()

    #Evaluate the reply using Groq
    evaluation = evaluate(reply, message, history)

    #Retry if not acceptable
    if evaluation.is_acceptable:
        print("Feedback:", evaluation.feedback)
    else:
        print("Failed evaluation - retrying")
        print("Feedback:", evaluation.feedback)
        reply = rerun(reply, message, evaluation.feedback, system,model)

    return reply


In [209]:
# gr.ChatInterface(
#     fn=chat,  # your Gemini-Groq function
#     type="messages",  # use OpenAI-style messages
#     chatbot=gr.Chatbot(height=400),  # NO `type` here!
#     title="Gemini × Groq Self-Evaluating Chatbot"
# ).launch()
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7892
* To create a public link, set `share=True` in `launch()`.


Failed evaluation - retrying
Feedback: The response is in Pig Latin, which is acceptable, but it does not provide a clear answer to the user's question of whether Ansh holds a patent. A more informative response is needed.
